In [1]:
from math_go_brrr import brrr
from time import time
from types import FunctionType

## Comparing the Python and LLVM JIT version

In [4]:
@brrr
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b



start = time()
llvm_out = foo(10_000_000)
print(f"LLVM JIT Compiled |", time() - start)

# foo.original_func is our plain-old Python function
assert isinstance(foo.original_func, FunctionType)

start = time()
python_out = foo.original_func(10_000_000)
print(f"Regular Python    |", time() - start)

LLVM JIT Compiled | 0.02709507942199707
Regular Python    | 0.4261910915374756


## Testing various optimization levels

In [11]:
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b


for lvl in ["none", "less", "default", "aggressive"]:
    foo_compiled = brrr(optimization=lvl)(foo)

    start = time()
    llvm_out = foo_compiled(10_000_000)
    print(f"LLVM opt = {lvl:<10} |", time() - start)

start = time()
python_out = foo(10_000_000)
print(f"Regular Python        |", time() - start)

LLVM opt = none       | 0.026751041412353516
LLVM opt = less       | 0.07647323608398438
LLVM opt = default    | 0.026397228240966797
LLVM opt = aggressive | 0.02455902099609375
Regular Python        | 0.3632378578186035


## Testing the performance with different input sizes

Since the calling of the JIT function jumps through a few more hoops internally and has to convert python types to LLVM types, the JIT version is slower at small input sizes. However, it scales much better than python with larger input sizes.

In [21]:
@brrr
def foo(a: int) -> int:
    b = 0
    i = 0
    while i < a:
        b = b + i
        i = i + 1

    return b

# the first run does very badly,
# and I assume because of the JIT it might
# only compile when it's called the first time
foo(1)

for count in [1, 10, 100, 10_000, 1_000_000]:
  print(f"Count: {count}")
  start = time()
  llvm_out = foo(count)
  print(f"  LLVM JIT Compiled | {time() - start:.7f}")

  # foo.original_func is our plain-old Python function
  assert isinstance(foo.original_func, FunctionType)

  start = time()
  python_out = foo.original_func(count)
  print(f"  Regular Python    | {time() - start:.7f}")

Count: 1
  LLVM JIT Compiled | 0.0000341
  Regular Python    | 0.0000021
Count: 10
  LLVM JIT Compiled | 0.0000041
  Regular Python    | 0.0000012
Count: 100
  LLVM JIT Compiled | 0.0000041
  Regular Python    | 0.0000038
Count: 10000
  LLVM JIT Compiled | 0.0000279
  Regular Python    | 0.0003819
Count: 1000000
  LLVM JIT Compiled | 0.0024588
  Regular Python    | 0.0369978
